In [9]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import shooju
from shooju import points_serializers as ps
from datetime import datetime
import plotly.graph_objects as go
#!pip install dataframe_image
import dataframe_image as dfi
from datetime import date
from datetime import timedelta

## Corpus

In [18]:
# Get today's date and yesterday's date
today = date.today()
yesterday = today - timedelta(days = 1)
print(yesterday)
# Import data for prices

terminals = ['MEH','MIDSWT','WTICORPUS','WTIECHO','WTINEDER','WTS']
terminalnames = ['MEH','MIDSWT','WTI Corpus','WTI Echo','WTI Neder','WTS']
for x in (range(len(terminals))):
    sj = shooju.Connection(server="https://energyaspects.shooju.com",
                           user='debtanu.bandyopadhyay',
                           api_key='lOM08aUl15Si5wlUcnD5pym2NRp9bWUCybctuSzJFUx7NZ6IRJPMDIqlEKdPKudNl')

    sid= 'sid=users\\debtanu.bandyopadhyay\\link\\settle_prices\\'+terminals[x]+'\\'
    df_table=sj.get_series(sid, max_points=-1, serializer=ps.pd_series)
    df_table=pd.DataFrame.from_dict(df_table)

    # import data for MTD
    sj = shooju.Connection(server="https://energyaspects.shooju.com",
                           user='debtanu.bandyopadhyay',
                           api_key='lOM08aUl15Si5wlUcnD5pym2NRp9bWUCybctuSzJFUx7NZ6IRJPMDIqlEKdPKudNl')

    sid1='sid=users\\debtanu.bandyopadhyay\\link\\MTD\\'+terminals[x]+'\\'
    df_table_MTD=sj.get_series(sid1, max_points=-1, serializer=ps.pd_series)
    df_table_MTD=pd.DataFrame.from_dict(df_table_MTD)

    # Formatting the downloaded data into the wanted df format
    df_table.drop('series_id', axis = 1, inplace = True)
    df_table.rename(columns = {'points':'price'}, inplace = True)
    df_table.reset_index(inplace = True)
    df_table['month'] = df_table['index'].map(str)
    df_table['month'] = df_table['month'].str.extract(r'(\d+\D+\d+)', expand=False)
    df_table = df_table.set_index('month')

    df_table_MTD.drop('series_id', axis = 1, inplace = True)
    df_table_MTD.rename(columns = {'points':'MTD'}, inplace = True)
    df_table_MTD.reset_index(inplace = True)
    df_table_MTD['index'] = df_table_MTD['index'].map(str)
    df_table_MTD['index'] = df_table_MTD['index'].str.extract(r'(\d+\D+\d+)', expand=False)
    df_table_MTD = df_table_MTD.set_index('index')

    df = pd.merge(df_table_MTD, df_table, left_index = True, right_index = True)
    df['change'] = df['price'].diff(1)
    df = df.set_index('index')
    df['Previous'] = df['price'].shift()
    df.columns = ['MTD', 'Settle','Change','Previous']
    df = df.reindex(columns=['Settle','Previous', 'Change','MTD'])
    rows = df.loc[yesterday]
    df = pd.DataFrame(rows)
    df = df.reset_index()
    df.columns = [terminalnames[x],'']

    index_names = {
        "selector": ".index_name",
        "props": "font-style: italic; color: darkgrey; font-weight:normal;"
    }
    headers = {
        "selector": "th:not(.index_name)",
        "props": "background-color: #990033; color: white; text-align: Center",
        "border": "1px solid black",
        "width": "45px"
    }
    properties = {"border": "1px solid black", "width": "45px", "text-align": "center"}



    df = df.style.format(precision=2).set_table_styles([index_names, headers]).set_properties(**properties).hide(axis="index")



    display(df)
    dfi.export(df, terminalnames[x]+".png")


2022-09-14


KeyError: datetime.date(2022, 9, 14)

In [19]:

df

,Settle,Previous,Change,MTD
index,,,,
2017-02-01,1.70,NaN,NaN,1.71
2017-02-02,1.65,1.70,-0.05,1.71
2017-02-03,1.65,1.65,0.00,1.71
2017-02-06,1.70,1.65,0.05,1.71
2017-02-07,1.80,1.70,0.10,1.71
...,...,...,...,...
2022-09-08,2.25,2.30,-0.05,1.98
2022-09-09,2.25,2.25,0.00,1.98
2022-09-12,2.30,2.25,0.05,1.98


In [25]:
(datetime.now() - timedelta(1)).strftime('%Y-%m-%d')

'2022-09-14'

None